## Import

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import cv2

from PIL import Image
import thinplate as tps

In [ ]:
#large radius => brighter around center. Too small radius would cut the image
def create_grad_img(center, radius, thickness = 2, speed=1):
    grad_img = np.zeros((512, 512, 3), dtype='uint8')
    for i in range(1,radius):
        color = 255-int(speed*i/radius*255)
        grad_img = cv2.circle(grad_img, center, i, (color, color, color), thickness)
    return grad_img

def get_rotate_matrix(h, w, alpha, beta, gamma, dx, dy, dz, f):
    """
    Source: http://jepsonsblog.blogspot.com/2012/11/rotation-in-3d-using-opencvs.html
    90 degrees being the "normal" position.
    
    alpha: the rotation around the x axis
    beta: the rotation around the y axis
    gamma: the rotation around the z axis (basically a 2D rotation)
    dx: translation along the x axis
    dy: translation along the y axis
    dz: translation along the z axis (distance to the image)
    f: focal distance (distance between camera and image, a smaller number exaggerates the effect)
    """

    alpha = (alpha - 90.)*np.pi/180.
    beta = (beta - 90.)*np.pi/180.
    gamma = (gamma - 90.)*np.pi/180.
    # Projection 2D -> 3D matrix
    A1 = np.array([[1, 0, -w/2],
          [0, 1, -h/2],
          [0, 0,    0],
          [0, 0,    1]])
    # Rotation matrices around the X, Y, and Z axis
    RX = np.array([[1,          0,           0, 0],
          [0, np.cos(alpha), -np.sin(alpha), 0],
          [0, np.sin(alpha),  np.cos(alpha), 0],
          [0,          0,           0, 1]])
    RY = np.array([[np.cos(beta), 0, -np.sin(beta), 0],
          [0, 1,          0, 0],
          [np.sin(beta), 0,  np.cos(beta), 0],
          [0, 0,          0, 1]])
    RZ = np.array([[np.cos(gamma), -np.sin(gamma), 0, 0],
          [np.sin(gamma),  np.cos(gamma), 0, 0],
          [0,          0,           1, 0],
          [0,          0,           0, 1]])
    # Composed rotation matrix with (RX, RY, RZ)
    R = np.matmul(np.matmul(RX, RY), RZ)
    # Translation matrix
    T = np.array([[1, 0, 0, dx],
         [0, 1, 0, dy],
         [0, 0, 1, dz],
         [0, 0, 0, 1]])
    # 3D -> 2D matrix
    A2 = np.array([[f, 0, w/2, 0],
          [0, f, h/2, 0],
          [0, 0,   1, 0]])
    # Final transformation matrix
    return np.matmul(A2, np.matmul(T, np.matmul(R, A1)))

def first_nonzero(arr, axis, invalid_val=-1):
    mask = arr!=0
    return np.where(mask.any(axis=axis), mask.argmax(axis=axis), invalid_val)

def last_nonzero(arr, axis, invalid_val=-1):
    mask = arr!=0
    val = arr.shape[axis] - np.flip(mask, axis=axis).argmax(axis=axis) - 1
    return np.where(mask.any(axis=axis), val, invalid_val)

def get_mask_bbox(img):
    sum_y = np.sum(img, axis=0)
    sum_x = np.sum(img, axis=1)
    l = first_nonzero(sum_y, 0)[0]
    r = last_nonzero(sum_y, 0)[0]
    t = first_nonzero(sum_x, 0)[0]
    b = last_nonzero(sum_x, 0)[0]
    return (l, r, t, b)

def mask_to_shadow(mask, rot_rd_low, rot_rd_high, trans_rd_low=0, trans_rd_high=0):
    # Rotate image
    h, w = mask.shape[0], mask.shape[1]
    rot_rd = np.random.randint(low = rot_rd_low, high = rot_rd_high)
    rotate_matrix = get_rotate_matrix(h, w, alpha=90+rot_rd, beta=90, gamma=90, dx=0, dy=0, dz=200, f=200)
    rotated = cv2.warpPerspective(mask, rotate_matrix, (h, w), cv2.INTER_LANCZOS4)
    
    # Translate image so the shadow touch the legs
    # Translate after rotate x-axis or it won't be correct
    l, r, t, b = get_mask_bbox(mask)
    new_l, new_r, new_t, new_b = get_mask_bbox(rotated)
    # # matching the x-axis center of 2 bboxes
    # dx=int((r-l)-(new_r-new_l))//2+np.random.randint(low=trans_rd_low, high=trans_rd_high)
    # # matching the bottom of 2 bboxes
    # dy=int(b-new_b)+np.random.randint(low=trans_rd_low, high=trans_rd_high)
    # test Thin plate spline
    dx=0
    dy=0
    trans_matrix = get_rotate_matrix(h, w, alpha=90, beta=90, gamma=90, dx=dx, dy=dy, dz=200, f=200)
    warped = cv2.warpPerspective(rotated, trans_matrix, (h, w), cv2.INTER_LANCZOS4)
    return warped, rotate_matrix, trans_matrix

def create_gradient_shadow_alpha_mask(shadow_mask, rd_low, rd_high):
    """
    The rd_low and high determine the gradient shade of the shadow.
    Increase the value to have fader shade
    """
    final_l, final_r, final_t, final_b = get_mask_bbox(shadow_mask)
    rd = np.random.randint(low = rd_low, high = rd_high)
    grad_img = create_grad_img(((final_l+final_r)//2,final_b+rd),256) # Hard-code here

    # Normalize the alpha mask to keep intensity between 0 and 1
    alpha = grad_img.astype(float)/255
    return alpha

def convert_pts(src_shape, src):    
    src = np.squeeze(src, axis=0)
    c_src = np.zeros_like(src)
    c_src[:,0] = src[:,0]/src_shape[1]
    c_src[:,1] = src[:,1]/src_shape[0]
    return c_src

def show_warped(img, warped, c_src, c_dst):
    fig, axs = plt.subplots(1, 2, figsize=(16,8))
    axs[0].axis('off')
    axs[1].axis('off')
    axs[0].imshow(img[...,::-1], origin='upper')
    axs[0].scatter(c_src[:, 0]*img.shape[1], c_src[:, 1]*img.shape[0], marker='+', color='red')
    axs[1].imshow(warped[...,::-1], origin='upper')
    axs[1].scatter(c_dst[:, 0]*warped.shape[1], c_dst[:, 1]*warped.shape[0], marker='+', color='red')
    plt.show()

def warp_image_cv(img, c_src, c_dst, dshape=None):
    dshape = dshape or img.shape
    theta = tps.tps_theta_from_points(c_src, c_dst, reduced=True)
    grid = tps.tps_grid(theta, c_dst, dshape)
    mapx, mapy = tps.tps_grid_to_remap(grid, img.shape)
    return cv2.remap(img, mapx, mapy, cv2.INTER_CUBIC)

## Note:
- Images error: 17, 19, 12, etc (due to hardcode 512x512)

In [ ]:
tmp = "4"
img = cv2.imread(f'./svm_samples/images/{tmp}.png')
mask = cv2.imread(f'./svm_samples/foreground_object_mask/{tmp}.png')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
background=cv2.subtract(img,mask)
cut_obj=cv2.subtract(img,background)

In [ ]:
shadow_mask, rotate_matrix, trans_matrix = mask_to_shadow(mask, rot_rd_low=10, rot_rd_high=80)

old_points = np.float32([[
    [217, 319],#(217,319)
    [233, 314],#(233,314)
    [267, 340],#(267,340)
    [289, 334]#(289,334)
]])
rotated_points = cv2.perspectiveTransform(old_points, rotate_matrix)
translated_points = cv2.perspectiveTransform(rotated_points, trans_matrix)
conv_old_pts = convert_pts(shadow_mask.shape[:2], old_points)
conv_rotated_pts = convert_pts(shadow_mask.shape[:2], rotated_points)
conv_translated_pts = convert_pts(shadow_mask.shape[:2], translated_points)
warped_shadow_mask = warp_image_cv(shadow_mask, conv_translated_pts, conv_old_pts)

In [ ]:
alpha = create_gradient_shadow_alpha_mask(warped_shadow_mask, rd_low=0, rd_high=3)
revert_shadow_mask=cv2.subtract(img,warped_shadow_mask)

# Convert uint8 to float
# shadow_foreground = shadow_grad_blur.astype(float)
revert_shadow_mask_f = revert_shadow_mask.astype(float)
img_f = img.astype(float)

# Multiply the foreground with the alpha matte
foreground_shadow = cv2.multiply(alpha, revert_shadow_mask_f)

# Multiply the background with ( 1 - alpha )
background_img = cv2.multiply(1.0 - alpha, img_f)

# Add the masked foreground and background.
shadow_on_top = cv2.add(foreground_shadow, background_img)
shadow_on_top = shadow_on_top.astype('uint8')

final_background=cv2.subtract(shadow_on_top,mask)
final_img = cut_obj+final_background
plt.imshow(final_img[...,::-1])

## Notes:
- Control shadow rotation
- Extra pixels of the outline mask => ??? cut shadow mask, not object?